# Gold Layer Analytics — Answer Keys

**Schema**
- `gold.dim_customers`
- `gold.dim_products`
- `gold.fact_sales`

## Setup

In [ ]:
USE gold;

## 1) Oldest customer (by age)

In [ ]:
SELECT
  customer_key,
  customer_id,
  customer_number,
  first_name,
  last_name,
  birthdate,
  TIMESTAMPDIFF(YEAR, birthdate, CURDATE()) AS approx_age_years
FROM gold.dim_customers
WHERE birthdate IS NOT NULL
ORDER BY birthdate ASC
LIMIT 1;

## 2) Youngest customer (by age)


In [ ]:
SELECT
  customer_key,
  customer_id,
  customer_number,
  first_name,
  last_name,
  birthdate,
  TIMESTAMPDIFF(YEAR, birthdate, CURDATE()) AS approx_age_years
FROM gold.dim_customers
WHERE birthdate IS NOT NULL
ORDER BY birthdate DESC
LIMIT 1;

## 3) Revenue by category

In [ ]:
SELECT
  dp.category,
  ROUND(SUM(fs.sales_amount), 2) AS revenue
FROM gold.fact_sales fs
JOIN gold.dim_products dp
  ON fs.product_key = dp.product_key
GROUP BY dp.category
ORDER BY revenue DESC;

## 4) Units sold by country

In [ ]:
SELECT
  dc.country,
  SUM(fs.quantity) AS units_sold
FROM gold.fact_sales fs
JOIN gold.dim_customers dc
  ON fs.customer_key = dc.customer_key
GROUP BY dc.country
ORDER BY units_sold DESC;

## 5) 3 customers with the fewest orders (customers with ≥ 1 order)

In [ ]:
SELECT
  dc.customer_key,
  dc.customer_id,
  dc.customer_number,
  dc.first_name,
  dc.last_name,
  COUNT(DISTINCT fs.order_number) AS order_count
FROM gold.fact_sales fs
JOIN gold.dim_customers dc
  ON fs.customer_key = dc.customer_key
GROUP BY
  dc.customer_key, dc.customer_id, dc.customer_number, dc.first_name, dc.last_name
HAVING COUNT(DISTINCT fs.order_number) >= 1
ORDER BY order_count ASC, dc.customer_key
LIMIT 3;

## 6) Bottom 5 products by revenue

In [ ]:
SELECT
  dp.product_name,
  ROUND(SUM(fs.sales_amount), 2) AS revenue
FROM gold.fact_sales fs
JOIN gold.dim_products dp
  ON fs.product_key = dp.product_key
GROUP BY dp.product_name
ORDER BY revenue ASC
LIMIT 5;

## 7) Yearly product sales

In [ ]:
SELECT
  YEAR(fs.order_date) AS order_year,
  dp.product_name,
  ROUND(SUM(fs.sales_amount), 2) AS revenue
FROM gold.fact_sales fs
JOIN gold.dim_products dp
  ON fs.product_key = dp.product_key
GROUP BY YEAR(fs.order_date), dp.product_name
ORDER BY order_year, revenue DESC, dp.product_name;